In [1]:
!nvidia-smi

Sun Mar 23 15:02:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
!wget https://github.com/Subrat1920/IMDB-Sentiment-Analysis/raw/refs/heads/main/Notebook/keras_models/sentiment_model.keras

--2025-03-23 15:03:05--  https://github.com/Subrat1920/IMDB-Sentiment-Analysis/raw/refs/heads/main/Notebook/keras_models/sentiment_model.keras
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Subrat1920/IMDB-Sentiment-Analysis/refs/heads/main/Notebook/keras_models/sentiment_model.keras [following]
--2025-03-23 15:03:05--  https://raw.githubusercontent.com/Subrat1920/IMDB-Sentiment-Analysis/refs/heads/main/Notebook/keras_models/sentiment_model.keras
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40757004 (39M) [application/octet-stream]
Saving to: ‘sentiment_model.keras’

sentiment_model.ker 

In [4]:
from tensorflow.keras.models import load_model
model = load_model("sentiment_model.keras")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 300)            │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 500, 256)            │         330,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 500, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 64)                  │          61,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,176,389 (38.82 MB)

 Trainable params: 3,392,129 (12.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,784,260 (25.88 MB)

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [6]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key,value in word_index.items()}

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [16]:
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in encoded_review])

def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = pad_sequences([encoded_review], maxlen=500, padding='pre')
    return padded_review


def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0] > 0.70 else 'Negative'
    return sentiment, prediction[0][0]

In [17]:
example_1 = "This movie was fantastic! The acting was great and the plot was thrilling, A great movie requires great ideas."
example_2 = "This movie was amazing! The performances were stellar, and the storyline kept me on the edge of my seat. A truly great film is built on brilliant concepts and creativity."
example_3 = "The movie was bad, i did not like the movie, it was so predictable."
example_4 = "The movie was bad"
example_5 = "The moview was good"

In [20]:
predict_sentiment(example_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


('Positive', np.float32(0.98608834))

In [22]:
for i in range(1, 6):
    val = f"example_{i}"
    text = globals()[val]
    print(f"Predicting the Sentiment for: {text}")
    print('-' * 50)
    print(predict_sentiment(text))
    print('-' * 50)

Predicting the Sentiment for: This movie was fantastic! The acting was great and the plot was thrilling, A great movie requires great ideas.
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
('Positive', np.float32(0.98581123))
--------------------------------------------------
Predicting the Sentiment for: This movie was amazing! The performances were stellar, and the storyline kept me on the edge of my seat. A truly great film is built on brilliant concepts and creativity.
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
('Positive', np.float32(0.98608834))
--------------------------------------------------
Predicting the Sentiment for: The movie was bad, i did not like the movie, it was so predictable.
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
('Negative', np.float32(0.6070956))
--------------------------------------------------
Predicting the Sentiment for: 